In [54]:
import re

In [55]:
questions = [
    {
        'id': 0,
        'question': 'What is your name?',
        'attribute': 'name',
        'type': 'text',
        'conditions': None  # Always shown first
    },
    {
        'id': 1,
        'question': 'Which gender perfume are you looking for?',
        'type': 'multi-choice',
        'attribute': 'gender',
        'answers': {1: 'Women', 2: 'Men', 3: 'Unisex'},
        'conditions': None  
    },
    {
        'id': 2,
        'question': 'Are you shopping for yourself or buying a gift?',
        'type': 'single-choice',
        'attribute': 'target',
        'answers': {1: 'Myself', 2: 'Gift'},
        'conditions': None  
    },
    {
        'id': 3,
        'question': 'Which personality best describes the recipient?',
        'type': 'single-choice',
        'attribute': 'personality',
        'answers': {1: 'Adventurous / Bold', 2: 'Romantic / Elegant', 3: 'Fresh / Energetic', 4: 'Warm / Comforting', 5: 'Exotic / Mysterious'},
        'conditions': {'target': [2]}  
    },
    
    # Personality-specific questions
    {
        'id': 4,
        'question': 'What kind of bold scent would they prefer?',
        'type': 'multi-choice',
        'attribute': 'bold-scent',
        'answers': {1: 'Woody (strong, earthy)', 2: 'Spicy (warm, intense)', 3: 'Oriental (rich, exotic)'},
        'conditions': {'personality': [1]}  # Only if Adventurous/Bold
    },
    {
        'id': 5,
        'question': 'Which elegant note fits their style?',
        'type': 'multi-choice',
        'attribute': 'elegant-scent',
        'answers': {1: 'Floral and Oriental (classic, feminine)', 2: 'Sweet Aromatic (soft, charming)'},
        'conditions': {'personality': [2]}  # Only if Romantic/Elegant
    },
    {
        'id': 6,
        'question': 'Which fresh vibe suits them best?',
        'type': 'multi-choice',
        'attribute': 'fresh-scent',
        'answers': {1: 'Fresh (clean, crisp)', 2: 'Citrus (zesty, bright)', 3: 'Fruity (sweet, lively)'},
        'conditions': {'personality': [3]}  # Only if Fresh/Energetic
    },
    {
        'id': 7,
        'question': 'Which comforting scent would they enjoy?',
        'type': 'multi-choice',
        'attribute': 'comforting-scent',
        'answers': {1: 'Vanilla (sweet, cozy)', 2: 'Musk (soft, sensual)', 3: 'Sandalwood (smooth, woody)'},
        'conditions': {'personality': [4]}  # Only if Warm/Comforting
    },
    {
        'id': 8,
        'question': 'Which mysterious scent appeals to them?',
        'type': 'multi-choice',
        'attribute': 'mysterious-scent',
        'answers': {1: 'Woody and Spicy (bold, complex)', 2: 'Arabian (rich, luxurious)'},
        'conditions': {'personality': [5]}  # Only if Exotic/Mysterious
    },
    
    # Myself flow
    {
        'id': 9,
        'question': 'Which season do you prefer the perfume for?',
        'type': 'single-choice',
        'attribute': 'season',
        'answers': {1: 'Spring / Summer', 2: 'Fall / Winter'},
        'conditions': {'target': [1]}  
    },
        
        # Occasion questions
    {
        'id': 10,
        'question': 'What occasion is the perfume for?',
        'type': 'single-choice',
        'attribute': 'occasion',
        'answers': {1: 'Casual', 2: 'Formal', 3: 'Romantic', 4: 'Party', 5: 'Work'},
        'conditions': {'season': [1]} 
    },
    {
        'id': 11,
        'question': 'What occasion is the perfume for?',
        'type': 'single-choice',
        'attribute': 'occasion',
        'answers': {1: 'Casual', 2: 'Formal', 3: 'Romantic', 4: 'Party', 5: 'Work'},
        'conditions': {'season': [2]} 
    },
        # Preferred scents (Myself route)
    {
        'id': 12,
        'question': 'Which fresh vibe suits them best?',
        'type': 'single-choice',
        'attribute': 'fresh-scent-self',
        'answers': {1: 'Fresh (clean, crisp)', 2: 'Citrus (zesty, bright)', 3: 'Fruity (sweet, lively)'},
        'conditions': {'season': [1], 'occasion': [1, 4]}  
    },
    {
        'id': 13,
        'question': 'Which elegant note fits your style?',
        'type': 'multi-choice',
        'attribute': 'elegant-scent-self',
        'answers': {1: 'Floral and Oriental (classic, feminine)', 2: 'Sweet Aromatic (soft, charming)'},
        'conditions': {'season': [1], 'occasion': [2, 3]}  
    },
    {
        'id': 14,
        'question': 'Which clean or aromatic scent do you prefer?',
        'type': 'multi-choice',
        'attribute': 'clean-aromatic-scent-self',
        'answers': {1: 'Clean (fresh, crisp)', 2: 'Aromatic (herbal, lively)', 3: 'Woody (warm, classic)'},
        'conditions': {'season': [1, 2], 'occasion': [5]}  
    },
    {
        'id': 15,
        'question': 'What kind of bold scent do you prefer?',
        'type': 'multi-choice',
        'attribute': 'bold-scent-self',
        'answers': {1: 'Woody (strong, earthy)', 2: 'Spicy (warm, intense)', 3: 'Oriental (rich, exotic)'},
        'conditions': {'season': [2], 'occasion': [1, 4]}  
    },
    {
        'id': 16,
        'question': 'Which comforting scent would you enjoy?',
        'type': 'multi-choice',
        'attribute': 'comforting-scent-self',
        'answers': {1: 'Vanilla (sweet, cozy)', 2: 'Musk (soft, sensual)', 3: 'Sandalwood (smooth, woody)'},
        'conditions': {'season': [2], 'occasion': [2, 3]}
    },
    
    
    # Brand category question
    {
        'id': 17,
        'question': 'Please choose your preferred brand category?',
        'type': 'single-choice',
        'attribute': 'brand-category',
        'answers': {
            1: 'Luxury: (Chanel, Dior, Gucci, etc.)',
            2: 'Mid-Range: (Hugo Boss, Calvin Klein, Marc Jacobs, etc.)',
            3: 'Designer: (Versace, Michael Kors, Coach, etc.)',
            4: 'Niche: (Arabian Oud, Rasasi, Ajmal, etc.)',
            5: 'Affordable: (Avon, Revlon, Elizabeth Arden, etc.)'
        },
        'conditions': None  
    },
    
    # Budget questions (conditional based on brand category)
    {
        'id': 18,
        'question': "What's your budget range for the perfume?",
        'type': 'single-choice',
        'attribute': 'budget-luxury',
        'answers': {1: '$100 - $200', 2: '$200 - $300', 3: '$300+'},
        'conditions': {'brand-category': [1, 4]}  # For Luxury and Niche
    },
    {
        'id': 19,
        'question': "What's your budget range for the perfume?",
        'type': 'single-choice',
        'attribute': 'budget-midrange', 
        'answers': {1: '$50 - $100', 2: '$100 - $150', 3: '$150+'},
        'conditions': {'brand-category': [2, 3]}  # For Mid-Range and Designer
    },
    {
        'id': 20,
        'question': "What's your budget range for the perfume?",
        'type': 'single-choice',
        'attribute': 'budget-affordable',  
        'answers': {1: 'Under $50', 2: '$50 - $75', 3: '$75+'},
        'conditions': {'brand-category': [5]}  # For Affordable
    } 
]

In [56]:
# Check input from user and display error if user chose invalid option 
def check_in_valid_range(user_input_list, valid_range):
    # Check if user_input_list is empty
    if not user_input_list:
        print("Invalid choice (empty).")
        return False
    
    for user_input in user_input_list:
        try:
            num = int(user_input)  # Try converting to integer
            if 1 <= num <= valid_range: # Check if it's within the valid range
                continue
            else:
                print(f"Invalid choice. Please input a number within the valid range.")
                return False
        except ValueError:
            # If it's not a valid integer
            print(f"Invalid input '{user_input}'. Please input a whole number.")
            return False
    
    return True

In [57]:
def multi_choice_question(question_id):
    q = questions[question_id]
    valid_range = len(q['answers'])
    
    print(q['question'])
    
    # Save and only print answer options for type "multi-choice" questions
    user_input=''
    print("Options: ")
    for idx, key in q['answers'].items():
        print(f"{idx}. {key}")
                
    # Keep asking until valid
    while True:
        user_input = input(
            f"Input a number from 1 to {valid_range}"
            "(e.g: 1, 2)"
        ).strip()
        
        print(f"You input: {user_input}")
        
        user_input_list = [(x) for x in re.split(r'[,\s;]+', user_input) if x]

        if check_in_valid_range(user_input_list, valid_range):
            break
        
    # Update boolean flags
    result = {}
    
    for answer_key, answer_value in q['answers'].items():
        if str(answer_key) in user_input_list:
            result[answer_value] = True
        else:
            result[answer_value] = False
                
    # Save in user profile
    user_profile[q['attribute']] = result
    
    return user_input_list

In [58]:
def single_choice_question(question_id):
    q = questions[question_id]
    valid_range = len(q['answers'])

    print(q['question'])

    # Print answer options
    user_input = ''
    print("Options: ")
    for idx, key in q['answers'].items():
        print(f"{idx}. {key}")

    # Keep asking until valid
    while True:
        user_input = input(f"Select one option from these (e.g: 1): ").strip()
        
        print(f"You input: {user_input}")

        user_input_list = [(x) for x in re.split(r'[,\s;]+', user_input) if x]
        
        if len(user_input_list) != 1:
            print("Invalid choice. Please select ONE option")
            continue
            
        if check_in_valid_range(user_input_list, valid_range):
            break
        
    # Update boolean flags based on the selected answer
    result = {}
    for answer_key, answer_value in q['answers'].items():
        # Compare the string input directly with the answer key
        if str(answer_key) == user_input:
            result[answer_value] = True
        else:
            result[answer_value] = False

    # Save in user profile
    user_profile[q['attribute']] = result

    # Return the selected option for routing or further logic
    return int(user_input)

In [59]:
def text_question(question_id):
    q = questions[question_id]
    print(q['question'])
    user_input = str(input(f"Your name: "))
    print(f"Your name: {user_input}")
    user_profile[q['attribute']] = user_input

In [60]:
def ask_question(question_id):
    q = questions[question_id]
    if q['type'] == 'multi-choice':
        return multi_choice_question(question_id)
    elif q['type'] == 'single-choice':
        return single_choice_question(question_id)
    elif q['type'] == 'text':
        return text_question(question_id)

In [61]:
user_profile = {}

In [62]:
def main():
    print("Welcome to the Fragrance Recommendation System!")

    # Q0: Name of user
    ask_question(0)
    
    # Q1: Gender (multi-choice)
    ask_question(1)
    
    # Q2: Target (single choice - myself or gift)
    
    target_choice = ask_question(2)
    
    # Branch based on target choice
    if target_choice == 1: #Myself flow
        print("\n-------- Shopping for yourself ---------\n")   
        
        # Q9: Season 
        season_choice = ask_question(9)
        
        # Q10 or Q11: Occasion (based on season)
        if season_choice == 1: # Spring/Summer
            occasion_choice = ask_question(10)
        else: # Fall/Winter
            occasion_choice = ask_question(11)
            
        # Additional scent questions based on season and occasion
        if season_choice == 1: # Spring/Summer
            if occasion_choice in [1, 4]: # Casual or Party
                ask_question(12) # Fresh scent
            elif occasion_choice in [2, 3]: # Formal or Romantic
                ask_question(13) # Elegant scent
            elif occasion_choice == 5:
                ask_question(14) # Clean/Aromatic
        else: # Fall/Winter
            if occasion_choice in [1, 4]: # Casual or Party
                ask_question(15) # Bold scent
            elif occasion_choice in [2, 3]: # Formal or Romantic
                ask_question(16) # Comforting scent 
            elif occasion_choice == 5: # Work
                ask_question(14)
                
    elif target_choice == 2: #Gift flow
        print("\n-------- Shopping for a gift ---------\n")   
        
        # Q3: Personality
        personality_choice = ask_question(3)
        
        # Personality-specific scent questions
        if personality_choice == 1: # Adventurous/Bold
            ask_question(4)
        elif personality_choice == 2: # Romantic/Elegant
            ask_question(5)
        elif personality_choice == 3:
            ask_question(6)
        elif personality_choice == 4:
            ask_question(7)
        elif personality_choice == 5:
            ask_question(8)
    
    # Mutual questions for both routes
    print("\n-------- Brand and Budget Information --------")
    
    # Q17: Brand category
    brand_choice = ask_question(17)
    
    # Budget question based on brand category
    if brand_choice in [1, 4]: # Luxury or Niche
        ask_question(18)
    elif brand_choice in [2, 3]:
        ask_question(19)
    elif brand_choice == 5: # Affordable
        ask_question(20)
        
    print(f"\nYour Profile Summary: ")
    for key, value in user_profile.items():
        if key != 'name':
            for k, v in value.items():
                if v == True:
                    print(f"{key}: {k}")
        else:
            print(f"{key}: {value}")
    return

In [63]:
if __name__ == "__main__":
    main()

Welcome to the Fragrance Recommendation System!
What is your name?
Your name: ellie
Which gender perfume are you looking for?
Options: 
1. Women
2. Men
3. Unisex
You input: 1
Are you shopping for yourself or buying a gift?
Options: 
1. Myself
2. Gift
You input: 1

-------- Shopping for yourself ---------

Which season do you prefer the perfume for?
Options: 
1. Spring / Summer
2. Fall / Winter
You input: 1
What occasion is the perfume for?
Options: 
1. Casual
2. Formal
3. Romantic
4. Party
5. Work
You input: 1
Which fresh vibe suits them best?
Options: 
1. Fresh (clean, crisp)
2. Citrus (zesty, bright)
3. Fruity (sweet, lively)
You input: 1

-------- Brand and Budget Information --------
Please choose your preferred brand category?
Options: 
1. Luxury: (Chanel, Dior, Gucci, etc.)
2. Mid-Range: (Hugo Boss, Calvin Klein, Marc Jacobs, etc.)
3. Designer: (Versace, Michael Kors, Coach, etc.)
4. Niche: (Arabian Oud, Rasasi, Ajmal, etc.)
5. Affordable: (Avon, Revlon, Elizabeth Arden, etc.)
Yo